# Should change Tkf below

In [2]:
import importlib
import time
import os
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import integrate
from scipy import interpolate

# Should change Tkf below

In [3]:
from multi_loc import covariance, assimilate, utilities

In [4]:
%matplotlib inline

In [5]:
def imshow(X, title=None, vmin=None, vmax=None, cmap=None):
    plt.figure()
    im = plt.imshow(X, vmin=vmin, vmax=vmax, cmap=cmap)
    plt.colorbar(im)
    plt.title(title)

In [6]:
tune_name = 'e2'

save_dir = 'dt_05_2'

home = os.path.expanduser('~')

save_dir = os.path.join(
    home,
    'Documents/python_code/multi_loc/notebooks/',
    save_dir)
    
this_path = os.path.join(save_dir, 'Z_GT.npy')
Zgt = np.load(this_path)

this_path = os.path.join(save_dir, 'Z0_ens.npy')
Z0ens = np.load(this_path)

this_path = os.path.join(save_dir, 't_gt.npy')
t_gt = np.load(this_path)

this_path = os.path.join(save_dir, 't_lr.npy')
t_lr = np.load(this_path)

this_path = os.path.join(save_dir, 'run_vars.pkl')
with open(this_path, mode='rb') as f:
    run_vars = pickle.load(f)

In [7]:
for key, value in run_vars.items():
    print(key + ' = ' + str(value))
    exec(key + '=value')

Nz = 960
K = 32
I = 12
F = 14.0
b = 1.0
c = 0.37
alpha = 0.12414383561643835
beta = 0.013746194824961947
coarse = 8
Nez = 20
Nex = 500
dt_rk = 0.0020833333333333333
dt_lr = 0.1
dt_gt = 0.0020833333333333333
Tic = 24.0
Tlr = 4000
Tgt = 250


# Should change Tkf below

In [8]:
tune_dir = os.path.join(save_dir, tune_name)
# if not os.path.exists(tune_dir):
#     os.mkdir(tune_dir)

# dt_trial_rk = 0.05/24
# dt_kf = 0.05
# dt_save = None
# Tkf = Tgt/5     ######## Change this (I forgot) #######


# alpha_range = np.array([0, 0.01, 0.02, 0.04, 0.08], dtype='float')
# loc_range = np.array([5, 10, 15, 20, 30, 40, 50, 60], dtype='float')


# R_sd = 2
# everyH = 24
# Hz = np.eye(Nz)[::everyH]
# Ny = Hz.shape[0]
# Rz = np.eye(Ny) * R_sd**2

# fine_obs_dict = {'dt_trial_rk': dt_trial_rk,
#                  'dt_kf': dt_kf,
#                  'dt_save': dt_save,
#                  'Tkf': Tkf,
#                  'Tspin': Tspin,
#                  'alpha_range': alpha_range,
#                  'loc_range': loc_range,
#                  'R_sd': R_sd,
#                  'everyH': everyH,
#                  'Hz': Hz,
#                  'Rz': Rz}

this_path = os.path.join(tune_dir, 'coarse_obs_dict.pkl')
with open(this_path, mode='rb') as f:
    fine_obs_dict = pickle.load(f)

In [9]:
for key, value in fine_obs_dict.items():
    print(key + ' = ' + str(value))
    exec(key + '=value')

dt_trial_rk = 0.0020833333333333333
dt_kf = 0.05
dt_save = None
Tkf = 50.0
Tspin = 25.0
alpha_range = [0.   0.05 0.1  0.15 0.2  0.25 0.3 ]
loc_range = [ 2.  5. 10. 15. 20. 25. 30. 35. 40.]
R_sd = 0.3
everyH = 2
Hz = [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Rz = [[0.09 0.   0.   ... 0.   0.   0.  ]
 [0.   0.09 0.   ... 0.   0.   0.  ]
 [0.   0.   0.09 ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.09 0.   0.  ]
 [0.   0.   0.   ... 0.   0.09 0.  ]
 [0.   0.   0.   ... 0.   0.   0.09]]


In [10]:
Nobs = int(Tkf/dt_kf) + 1
t_kf = np.linspace(0, Tkf, Nobs)

Ngt = int(Tgt/dt_gt) + 1
t_gt = np.linspace(0, Tgt, Ngt)
index_Tkf = int(Tkf/dt_gt) + 1

every_t = int(dt_kf/dt_gt)

if not np.allclose(t_kf, t_gt[0:index_Tkf:every_t]):
    print('They do not match')

In [11]:
Zloc = np.arange(Nz)

Zgt_ts = xr.DataArray(Zgt[:, 0:index_Tkf:every_t],
                      dims=('loc', 'time'),
                      coords={'loc': Zloc,
                              'time': t_kf})
this_path = os.path.join(tune_dir, 'Zobs_ts.nc')
Zobs_ts = xr.open_dataarray(this_path)
del Zgt

In [12]:
smooth_range = np.array([4, 8, 16])
coarse_loc_range = np.array([10, 20, 30, 40])
fine_loc_range = np.array([2, 4, 6, 8])
this_alpha = 0.05
alpha_count = 2
N_laml = 40

In [13]:
# smooth_range = np.array([4, 8, 16])
smooth_range = np.array([12, 8, 16, 4])
# coarse_loc_range = np.array([10, 20, 30, 40])
coarse_loc_range = np.array([30, 40, 20, 10])
# fine_loc_range = np.array([2, 4, 6, 8])
fine_loc_range = np.array([12, 10, 8, 6, 4, 2])
this_alpha = 0.025
alpha_count = 4
N_laml = 40

In [14]:
Tspin_index = np.argmin(np.abs(t_kf - Tspin))

In [15]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()

for smooth_count, this_smooth in enumerate(smooth_range):
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)
            
            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'infl': this_alpha,
                          'K': K,
                          'I': I,
                          'F': F,
                          'b': b,
                          'c': c,
                          'alpha': alpha,
                          'beta': beta}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('std_f: ', std_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
                print('std_a: ', std_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.025 ; smooth:  12
fine_loc:  12 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0000  has failed.
9.4 minutes

alpha:  0.025 ; smooth:  12
fine_loc:  12 ; coarse_loc:  40
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0001  has failed.
9.12 minutes

alpha:  0.025 ; smooth:  12
fine_loc:  12 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0000_floc_0000_cloc_0002  has failed.
9.34 minutes

alpha:  0.025 ; smooth:  12
fine

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0000_cloc_0001  has failed.
5.54 minutes

alpha:  0.025 ; smooth:  8
fine_loc:  12 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0000_cloc_0002  has failed.
5.58 minutes

alpha:  0.025 ; smooth:  8
fine_loc:  12 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0000_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0000_cloc_0003  has failed.
5.56 minutes

alpha:  0.025 ; smooth:  8
fine_loc:  10 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0001_floc_0001_cloc_0000
/Users/harty/Documents/python_code

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0000_cloc_0003  has failed.
5.58 minutes

alpha:  0.025 ; smooth:  16
fine_loc:  10 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0001_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0001_cloc_0000  has failed.
5.44 minutes

alpha:  0.025 ; smooth:  16
fine_loc:  10 ; coarse_loc:  40
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0001_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0001_cloc_0001  has failed.
5.53 minutes

alpha:  0.025 ; smooth:  16
fine_loc:  10 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0002_floc_0001_cloc_0002
/Users/harty/Documents/python_c

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0001_cloc_0001  has failed.
5.47 minutes

alpha:  0.025 ; smooth:  4
fine_loc:  10 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0001_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0001_cloc_0002  has failed.
5.5 minutes

alpha:  0.025 ; smooth:  4
fine_loc:  10 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0001_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0001_cloc_0003  has failed.
5.54 minutes

alpha:  0.025 ; smooth:  4
fine_loc:  8 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0004_smooth_0003_floc_0002_cloc_0000
/Users/harty/Documents/python_code/m

In [ ]:
#Bonus

In [16]:
smooth_range = np.array([4, 8, 16, 12])
coarse_loc_range = np.array([10, 20, 30, 40])
# fine_loc_range = np.array([2, 4, 6, 8])
this_alpha = 0.05
alpha_count = 2
N_laml = 40

fine_loc_count_add = 4
fine_loc_range = np.array([10, 12])

In [ ]:
rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())


rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])

In [17]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()
for smooth_count, this_smooth in enumerate(smooth_range):
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        fine_loc_count += fine_loc_count_add
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)
            
            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'infl': this_alpha,
                          'K': K,
                          'I': I,
                          'F': F,
                          'b': b,
                          'c': c,
                          'alpha': alpha,
                          'beta': beta}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.05 ; smooth:  4
fine_loc:  10 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0000  has failed.
5.68 minutes

alpha:  0.05 ; smooth:  4
fine_loc:  10 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0001  has failed.
5.61 minutes

alpha:  0.05 ; smooth:  4
fine_loc:  10 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0000_floc_0004_cloc_0002  has failed.
5.69 minutes

alpha:  0.05 ; smooth:  4
fine_loc:  

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0004_cloc_0001  has failed.
5.51 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  10 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0004_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0004_cloc_0002  has failed.
5.47 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  10 ; coarse_loc:  40
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0004_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0004_cloc_0003  has failed.
5.49 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  12 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0005_cloc_0000
/Users/harty/Documents/python_code

In [18]:
# smooth_range = np.array([4, 8, 16, 12])
coarse_loc_range = np.array([10, 20, 30, 40])
fine_loc_range = np.array([2, 4, 6, 8])
this_alpha = 0.05
alpha_count = 2
N_laml = 40

smooth_range = np.array([12])
smooth_count_add = 3

In [19]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()
for smooth_count, this_smooth in enumerate(smooth_range):
    smooth_count += smooth_count_add
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)
            
            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'infl': this_alpha,
                          'K': K,
                          'I': I,
                          'F': F,
                          'b': b,
                          'c': c,
                          'alpha': alpha,
                          'beta': beta}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.05 ; smooth:  12
fine_loc:  2 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0000  has failed.
5.1 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  2 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0001  has failed.
5.08 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  2 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/e2/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0002  has failed.
5.07 minutes

alpha:  0.05 ; smooth:  12
fine_loc:  

In [14]:
smooth_range = np.array([4, 8, 16])
coarse_loc_range = np.array([10, 20, 30, 40])
fine_loc_range = np.array([2, 4, 6, 8])

0.9040205185082489

In [ ]:
this_path = os.path.join(smooth_dir, 'rmse_array_f.npy')
np.save(this_path, rmse_array_f)

this_path = os.path.join(smooth_dir, 'std_array_f.npy')
np.save(this_path, std_array_f)

this_path = os.path.join(smooth_dir, 'rmse_array_a.npy')
np.save(this_path, rmse_array_a)

this_path = os.path.join(smooth_dir, 'std_array_a.npy')
np.save(this_path, std_array_a)

In [12]:
# Bonus

In [19]:
smooth_range = np.array([2])
smooth_count_add = 3
coarse_loc_range = np.array([10, 20, 30, 40])
fine_loc_range = np.array([2, 4, 6, 8])
this_alpha = 0.0
alpha_count = 1
N_laml = 40

In [21]:
smooth_range = np.array([2, 4, 8, 16], dtype='float')
coarse_loc_range = np.array([10, 20, 30, 40], dtype='float')
fine_loc_range = np.array([2, 4, 6, 8], dtype='float')
this_alpha = 0.05
alpha_count = 2
N_laml = 40

In [22]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()

for smooth_count, this_smooth in enumerate(smooth_range):
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)
            
            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'infl': this_alpha,
                          'K': K,
                          'I': I,
                          'F': F,
                          'b': b,
                          'c': c,
                          'alpha': alpha,
                          'beta': beta}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.05 ; smooth:  2.0
fine_loc:  2.0 ; coarse_loc:  10.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0000  has failed.
8.52 minutes

alpha:  0.05 ; smooth:  2.0
fine_loc:  2.0 ; coarse_loc:  20.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0001  has failed.
8.57 minutes

alpha:  0.05 ; smooth:  2.0
fine_loc:  2.0 ; coarse_loc:  30.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0002  has fa

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0000  has failed.
8.82 minutes

alpha:  0.05 ; smooth:  4.0
fine_loc:  6.0 ; coarse_loc:  20.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0001  has failed.
8.75 minutes

alpha:  0.05 ; smooth:  4.0
fine_loc:  6.0 ; coarse_loc:  30.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0002  has failed.
8.76 minutes

alpha:  0.05 ; smooth:  4.0
fine_loc:  6.0 ; coarse_loc:  40.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0001_

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0001  has failed.
8.64 minutes

alpha:  0.05 ; smooth:  16.0
fine_loc:  2.0 ; coarse_loc:  30.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0002  has failed.
8.61 minutes

alpha:  0.05 ; smooth:  16.0
fine_loc:  2.0 ; coarse_loc:  40.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0003  has failed.
8.6 minutes

alpha:  0.05 ; smooth:  16.0
fine_loc:  4.0 ; coarse_loc:  10.0
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05_2/dense_obs/smooth_loc/inf_0002_smooth_000

In [24]:
smooth_range = np.array([None])
coarse_loc_range = np.array([10, 20, 30, 40])
fine_loc_range = np.array([2, 4, 6, 8])
this_alpha = 0.1
alpha_count = 0
N_laml = 40

In [25]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()

for smooth_count, this_smooth in enumerate(smooth_range):
    smooth_count += 999
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)

            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'alpha': this_alpha}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.1 ; smooth:  None
fine_loc:  2 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0000  has failed.
11.9 minutes

alpha:  0.1 ; smooth:  None
fine_loc:  2 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0001  has failed.
17.1 minutes

alpha:  0.1 ; smooth:  None
fine_loc:  2 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0000_smooth_0999_floc_0000_cloc_0002  has failed.
17.7 minutes

alph

In [26]:
smooth_range = np.array([2, 4, 8, 16])
fine_loc_range = np.array([2, 4, 6, 8])
coarse_loc_range = np.array([10, 20, 30, 40])
this_alpha = 0.3
alpha_count = 2
N_laml = 40

In [ ]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([smooth_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)
time0 = time.time()

for smooth_count, this_smooth in enumerate(smooth_range):
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)

            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'alpha': this_alpha}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')
            time1 = time.time()
            print(f'{((time1 - time0)/60):0.3} minutes')
            print()
            time0 = time1

alpha:  0.3 ; smooth:  2
fine_loc:  2 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0000  has failed.
17.1 minutes

alpha:  0.3 ; smooth:  2
fine_loc:  2 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0001
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0001  has failed.
17.0 minutes

alpha:  0.3 ; smooth:  2
fine_loc:  2 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0000_floc_0000_cloc_0002  has failed.
17.0 minutes

alpha:  0.3 ;

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0001  has failed.
30.1 minutes

alpha:  0.3 ; smooth:  4
fine_loc:  6 ; coarse_loc:  30
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0002
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0002  has failed.
30.5 minutes

alpha:  0.3 ; smooth:  4
fine_loc:  6 ; coarse_loc:  40
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0002_cloc_0003  has failed.
29.8 minutes

alpha:  0.3 ; smooth:  4
fine_loc:  8 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0001_floc_0003_cloc_0000
/Users/harty/

/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0002  has failed.
30.4 minutes

alpha:  0.3 ; smooth:  16
fine_loc:  2 ; coarse_loc:  40
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0003
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0000_cloc_0003  has failed.
29.3 minutes

alpha:  0.3 ; smooth:  16
fine_loc:  4 ; coarse_loc:  10
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0001_cloc_0000
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0001_cloc_0000  has failed.
29.9 minutes

alpha:  0.3 ; smooth:  16
fine_loc:  4 ; coarse_loc:  20
/Users/harty/Documents/python_code/multi_loc/notebooks/dt_05/dense_obs/smooth_loc/inf_0002_smooth_0003_floc_0001_cloc_0001
/Users/har